<h1><center>Q-learning в средах OpenAI.gym</center></h1>

### Евгений Пономарев

### Сколковский институт науки и технологий

Библиотеки:
* `pip install numpy`
* `pip install tensorflow`
* `pip install gym`
* `pip install keras`

ffmpeg
Документация к openai.gym:
* https://gym.openai.com/docs/

## Обучение с подкреплением aka Reinforcement learning:
![](https://keras.io/img/keras-logo-small-wb.png)
* Библиотека для машинного обучения (прежде всего, обучения нейронных сетей, в т.ч. глубоких). 
* Представляет собой удобную обертку для мощных и хорошо оптимизированных вычислительных библиотек: TensorFlow, Theano
* Основные принципы: 
    1. Удобство использования
    2. Модульность
    3. Масштабируемость
    4. Работа с Python
    
Инструмент с низким порогом входа, подходящий как продвинутым исследовалетям, так и энтузиастам.

## Keras: краткий обзор
![](https://keras.io/img/keras-logo-small-wb.png)
* Библиотека для машинного обучения (прежде всего, обучения нейронных сетей, в т.ч. глубоких). 
* Представляет собой удобную обертку для мощных и хорошо оптимизированных вычислительных библиотек: TensorFlow, Theano
* Основные принципы: 
    1. Удобство использования
    2. Модульность
    3. Масштабируемость
    4. Работа с Python
    
Инструмент с низким порогом входа, подходящий как продвинутым исследовалетям, так и энтузиастам.

## Возвращение MNIST. Сравнения
Давайте вернемся к старой доброй задаче распознавания рукописных цифр, чтобы посмотреть сколько занимает код для такой нейросети, написанный на keras. Кроме того, здесь же мы попробуем собрать простую сверточную сеть для улучшения результата.

### Классическая нейросеть с плотными (dense) слоями.

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
import keras.datasets

Using TensorFlow backend.


In [2]:
# Загрузка данных. В keras уже есть несколько популярных датасетов, которые можно легко загрузить. Давайте загрузим MNIST
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

# Подготовка датасета: нормализация значений на [0,1] и перевод признаков в one-hot формат
X_train, X_test = X_train/255, X_test/255
y_train, y_test = keras.utils.to_categorical(y_train, 10), keras.utils.to_categorical(y_test, 10)
input_size = X_train[0].shape

In [3]:
# Создание модели. Sequential здесь означает последовательный тип модели, в который мы добавляем слои друг за другом
model = Sequential()

# Добавляем в стек модели слой за слоем. Полносвязный, активация и т.д.
# Важно: в первом слое Sequential модели keras необходимо указать размерность входных данных.
model.add(Flatten(input_shape=input_size))
model.add(Dense(units=256, input_shape=input_size))
model.add(Activation('relu'))
model.add(Dense(units=10))
model.add(Activation('softmax'))

In [4]:
# После описания архитектуры необходимо скомпилировать модель, указав минимизируемую функцию потерь, 
# оптимизатор и попросив модель выводить точность работы на тестовой выорке в процессе обучения
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

# Тренировка с указанием данных, числа эпох и размера подвыборки
model.fit(X_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
60000/60000 [==============================] - 8s - loss: 0.6187 - acc: 0.8470     
Epoch 2/5
60000/60000 [==============================] - 8s - loss: 0.3332 - acc: 0.9079     
Epoch 3/5
60000/60000 [==============================] - 8s - loss: 0.2851 - acc: 0.9205     
Epoch 4/5
60000/60000 [==============================] - 8s - loss: 0.2539 - acc: 0.9290     
Epoch 5/5
60000/60000 [==============================] - 8s - loss: 0.2297 - acc: 0.9355     


In [5]:
# Проверим качество работы модели на тестовых данных. Выводится loss и точночть.
model.evaluate(X_test, y_test)

 9952/10000 [============================>.] - ETA: 0s

[0.21461834994256496, 0.93879999999999997]

### Добавим сверточный слой и посмотрим на результат.

In [ ]:
from keras.layers import Conv2D
# Создание модели. Sequential здесь означает последовательный тип модели, в который мы добавляем слои друг за другом
conv_model = Sequential()

# Добавим явно число каналов в наш датасет - это важно для сверточных слоев. 
# т.е. делается преобразование (60000, 28, 28) -> (60000, 28, 28, 1). Это ничего не изменяет.
X_train, X_test = X_train.reshape((60000, 28, 28, 1)), X_test.reshape((10000, 28, 28, 1))
input_size = X_train[0].shape

# Здесь мы используем сверточный слой, который тренирует 32 фильтра размером 3x3 для поиска 
# конкретных геометрических (настраиваемых в процессе обучения)паттернов на входном изображении.
conv_model.add(Conv2D(24, (3, 3), input_shape=input_size))
conv_model.add(Activation('selu'))
conv_model.add(Flatten())
conv_model.add(Dense(64, activation='selu'))
conv_model.add(Dense(10, activation='softmax'))

In [ ]:
# Поставим другой оптимизатор для разнообразия
conv_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Тренировка с указанием данных, числа эпох и размера подвыборки
conv_model.fit(X_train[:30000], y_train[:30000], epochs=3, batch_size=32)

Epoch 1/3
10816/30000 [=========>....................] - ETA: 34s - loss: 0.4272 - acc: 0.8696

In [ ]:
# Проверим качество работы модели на тестовых данных. Выводится loss и точночть.
conv_model.evaluate(X_test, y_test)

## Сверточные слои, dropout


Генерируются и обучаются несколько фильтров небольших размеров так, чтобы распознавать какие то характерные сочетания пикселей (паттерны). Ниже на картинке изображение 5x5 пикселей, фильтр имеет размеры 3x3. При этом на выходе такой операции имеем картинку такого же размера, в каждый из пикселей которого записан результат свертки (число) данного фильтра с картинкой при нахождении центра фильтра в этом пикселе. Для этого исходную картинку необходимо дополнить по краям. Обычно это делают либо нулями, либо дублируют ближайшие пиксели (padding)
![](convol.gif)

Чем глубже сверточный слой, тем более сложные паттерны он способен распознавать:
![](features.png)

Dropout - техника спасения нейросетей от переобучения, при которой в процессе тренировки случайно "выключаются" некоторые нейроны из моделей.

Альтернативный взгляд - вместо тренировки одной большой сети проходит одновременная тренировка нескольких подсетей меньшего размера, результаты которых потом усредняются (в каком то смысле, сглаживаются).
![](dropout.gif)

Давайте попробуем посмотреть, как написать сеть, состоящую из нескольких сверточных слоев на `keras`

In [ ]:
from keras.layers import MaxPooling2D, Dropout

# Все так же, создаем модель
cnn = Sequential()

# Начинаем со сверточного слоя, указывая тип активации на выходе из него и способ заполнения краев (padding)
cnn.add(Conv2D(64, (3, 3), input_shape=input_size, activation='selu', padding='same'))

# Здесь мы используем метод MaxPooling, который уменьшает размер обрабатываемого изображения, 
# выбирая из 4 пикселей 1 с максимальным значением, чтобы это быстрее считалось. (2,2) -> 1
cnn.add(MaxPooling2D(pool_size=(2, 2)))

# Слой dropout, который на каждом шаге "выключает" 25% случайно выбранных нейронов
cnn.add(Dropout(0.25))

# Еще сверточный слой
cnn.add(Conv2D(32, (3, 3), input_shape=input_size, activation='selu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Dropout(0.5))

# Последний слой необходим для классификации, но перед ним необходимо векторизовать данные
cnn.add(Flatten())
cnn.add(Dense(10, activation='softmax'))


cnn.compile(loss='categorical_crossentropy',
                  optimizer = 'nadam',
                  metrics = ['accuracy'])

history_cnn = cnn.fit(X_train[:3000], y_train[:3000],
      batch_size=32,
      epochs=3,
      validation_data=(X_test, y_test))

## Дополнение данных в реальном времени
Большие модели требуют для обучения большого количества данных. Кроме того, иногда в задачах бывает крайне мало данных. В случае с изображениями в `keras` есть отличный инструмент для увеличения (раздувания) обучающей выборки.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot

# Задаем параметры, в рамках которых выборка может дополняться. Эти параметры сильно зависят от выборки.
# Например, в задаче распознавания рукописных цифр отражение изображения не особо релевантно, т.к. в тестовой выборке таких встретиться не может
shift = 0.1 # максимальное значени сдвига в долях (от 0 до 1)
angle = 15   # максимальный угол поворота 

# Команда создает генератор, который при вызове в режиме реального времени генерирует необходимую подвыборку нужного размера
datagen = ImageDataGenerator(width_shift_range=shift, 
                             height_shift_range=shift, 
                             rotation_range=angle, 
                             horizontal_flip=False, 
                             vertical_flip=False,
                             featurewise_center=True)

# Подстраиваем генератор под наши данные
datagen.fit(X_train)

# Выберем случайно 9 картинок дополненной выборки и нарисуем их
for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=9):
    # создаем сетку 3х3
    for i in range(0, 9):
        pyplot.subplot(330 + 1 + i)
        pyplot.imshow(X_batch[i].reshape(28, 28), cmap=pyplot.get_cmap('gray'))
    # рисуем картинки
    pyplot.show()
    break

Вообще, функция `ImageDataGenerator` - очень мощный инструмент препроцессинга. Например, он позволяет стандартизировать (нормализовать на среднее 0 и стандартное отклонение на 1) изображения попиксельно (достаточно укзать `featurewise_std_normalization=True` как её аргумент), а так же уменьшать избыточность матрицы изображений (`zca_whitening=True`)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot

# Задаем параметры, в рамках которых выборка может дополняться. Эти параметры сильно зависят от выборки.
# Например, в задаче распознавания рукописных цифр отражение изображения не особо релевантно, т.к. в тестовой выборке таких встретиться не может
shift = 0.1 # максимальное значени сдвига в долях (от 0 до 1)
angle = 15   # максимальный угол поворота 

# Команда создает генератор, который при вызове в режиме реального времени генерирует необходимую подвыборку нужного размера
datagen = ImageDataGenerator(width_shift_range=shift, 
                             height_shift_range=shift, 
                             rotation_range=angle, 
                             horizontal_flip=False, 
                             vertical_flip=False,
                             featurewise_std_normalization=True,
                             zca_whitening=True)

# Подстраиваем генератор под наши данные
datagen.fit(X_train)

# Выберем случайно 9 картинок дополненной выборки и нарисуем их
for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=9):
    # создаем сетку 3х3
    for i in range(0, 9):
        pyplot.subplot(330 + 1 + i)
        pyplot.imshow(X_batch[i].reshape(28, 28), cmap=pyplot.get_cmap('gray'))
    # рисуем картинки
    pyplot.show()
    break

Иногда возникает необходимость сохранить "раздутую" выборку. Для этого можно воспользоваться следующими командами:

In [ ]:
import os

# Задаем параметры, в рамках которых выборка может дополняться. Эти параметры сильно зависят от выборки.
# Например, в задаче распознавания рукописных цифр отражение изображения не особо релевантно, т.к. в тестовой выборке таких встретиться не может
shift = 0.1 # максимальное значени сдвига в долях (от 0 до 1)
angle = 15   # максимальный угол поворота 

# Команда создает генератор, который при вызове в режиме реального времени генерирует необходимую подвыборку нужного размера
datagen = ImageDataGenerator(width_shift_range=shift, 
                             height_shift_range=shift, 
                             rotation_range=angle, 
                             horizontal_flip=False, 
                             vertical_flip=False)

# Подстраиваем генератор под наши данные
datagen.fit(X_train)

# Создаем папку, куда сохраним изображения
os.makedirs('images')

# Выберем случайно 9 картинок дополненной выборки и нарисуем их
for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=9, save_to_dir='images', save_prefix='aug', save_format='png'):
    # создаем сетку 3х3
    for i in range(0, 9):
        pyplot.subplot(330 + 1 + i)
        pyplot.imshow(X_batch[i].reshape(28, 28), cmap=pyplot.get_cmap('gray'))
    # рисуем картинки
    pyplot.show()
    break

Общие советы по дополнению выборки:
* Смотрите на исходную выборку
* Смотрите на раздутую выборку
* Пробуйте разные трансформации, иногда неожиданные сочетания могут дать заметный прирост

## Transfer Learning (Fine Tuning) или зачем изобретать велосипед, когда можно встать на плечи гигантов?

Идея: взять уже натренированную на большом датасете большую нейросеть и переделать её под себя.

In [ ]:
# Несмотря на то, что мы не будем тренировать бОльшую часть модели, код ниже будет работать долго на любом cpu
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D

# Стандартизированный размер картинок для загружаемой сети. Это важный параметр! 
img_width, img_height = 150, 150

train_data_dir = 'cats_dogs/train'
validation_data_dir = 'cats_dogs/validation'
nb_train_samples = 10000
nb_validation_samples = 1000
epochs = 3
batch_size = 16

# Загружаем модель VCG16
base_model = applications.VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

# Создаем надстройку над ней для наших целей
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='selu')(x)
predictions = Dense(2, activation='softmax')(x)

# Сшиваем модели в одну
model = Model(inputs=base_model.input, outputs=predictions)

# Замораживаем (не тренируем) первые слои сети, чтобы тренировать лишь оставшиеся быстрее
for layer in base_model.layers:
    layer.trainable = False

# Здесь неплохо указать небольшой (на порядок или два меньше стандартного) шаг алгоритма оптимизации для более аккуратного поиска.
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-2, momentum=0.9),
              metrics=['accuracy'])

# Раздуваем данные
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# Дообучаем модель
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

# ``` Важно, изображения должны быть организованы следующим образом:
# data/
#     train/
#         dogs/
#             dog001.jpg
#             dog002.jpg
#             ...
#         cats/
#             cat001.jpg
#             cat002.jpg
#             ...
#     validation/
#         dogs/
#             dog001.jpg
#             dog002.jpg
#             ...
#         cats/
#             cat001.jpg
#             cat002.jpg
#             ...
# ```



In [ ]:
model.summary()

## Домашнее задание

0. [2] Поиграться с кодом, доступном в семинаре, подергать за ручки, подобавлять слои, поизменять dropout rate, batch_size, тип активации, количество фильтров свертки и т.д. посмотреть результат
0. [2] Загрузите доступный в `keras` датасет `cifar10`, постройте сверточную нейросеть, которая дает 70 % точности на тренировочной выборке. 80%? 90%?
0. [4] Построить на `mnist` сверточную нейросеть, в которой будет расти размер сверточных фильтров, но уменьшаться их число. Натренировать такую сеть. Нарисовать полученные натренированные фильтры, посмотреть, какие паттерны они распознают в цифрах.
0. [5] Попробуйте с помощью `keras` и нейросетей решить задачу регрессии (или классификации), не связанную с обработкой изображений: например, загрузите датасет `boston_housing`, [доступный](https://keras.io/datasets/) в `keras`. 

## Полезные материалы
Хочется отметить, что на русском языке материалов пока несравненно меньше, чем на английском:
* [Официальная документация](https://keras.io/) - библиотека отлично документирована
* [Keras в конкретных примерах](https://github.com/tmheo/keras_exercise) - 25 отличных jupyter notebooks
* [Упражнения и примеры в Keras и TensorFlow](https://github.com/leriomaggio/deep-learning-keras-tensorflow)

In [ ]:
# Stylish cell, better to compile at the beginning
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()


# from IPython.html.services.config import ConfigManager
# from IPython.utils.path import locate_profile
# cm = ConfigManager(profile_dir=locate_profile(get_ipython().profile))
# cm.update('livereveal', {
#               'fontsize': 4,
#               'theme': 'simple_cyr',
#               'transition': 'zoom',
#               'start_slideshow_at': 'selected',
#               'height': '724',
#               'scroll': True,
#               'slideNumber': True
# })

# Табличный Q-learning

Сопоставим каждой паре $(s,a)$ (состояние, действие) значение полезности - Q(s,a)

В простейшем случае, когда s~1,10,100; a~1,10 лучше просто выучить, что нужно делать, т.е. задать таблицу:

| a\s | $s_1$ | $s_2$  | $s_3$ | ... |
|-----|-----|------|-----|-----|
| $a_1$ | 1   | 0    | 8   | ... |
| $a_2$ | 5   | 4    | 3   | ... |
| $a_3$ | 8   | -10 | 99  | ... |

In [ ]:
import gym
import numpy
import random
import pandas

class QLearn:
    def __init__(self, actions, epsilon, alpha, gamma):
        self.q = {}
        self.epsilon = epsilon  # exploration constant
        self.alpha = alpha      # discount constant
        self.gamma = gamma      # discount factor
        self.actions = actions

    def getQ(self, state, action):
        return self.q.get((state, action), 0.0)

    def learnQ(self, state, action, reward, value):
        '''
        Q-learning:
            Q(s, a) += alpha * (reward(s,a) + max(Q(s') - Q(s,a))
        '''
        oldv = self.q.get((state, action), None)
        if oldv is None:
            self.q[(state, action)] = reward
        else:
            self.q[(state, action)] = oldv + self.alpha * (value - oldv)

    def chooseAction(self, state, return_q=False):
        q = [self.getQ(state, a) for a in self.actions]
        maxQ = max(q)

        if random.random() < self.epsilon:
            minQ = min(q); mag = max(abs(minQ), abs(maxQ))
            # add random values to all the actions, recalculate maxQ
            q = [q[i] + random.random() * mag - .5 * mag for i in range(len(self.actions))]
            maxQ = max(q)

        count = q.count(maxQ)
        # In case there're several state-action max values
        # we select a random one among them
        if count > 1:
            best = [i for i in range(len(self.actions)) if q[i] == maxQ]
            i = random.choice(best)
        else:
            i = q.index(maxQ)

        action = self.actions[i]
        if return_q: # if they want it, give it!
            return action, q
        return action

    def learn(self, state1, action1, reward, state2):
        maxqnew = max([self.getQ(state2, a) for a in self.actions])
        self.learnQ(state1, action1, reward, reward + self.gamma*maxqnew)

def build_state(features):
    return int("".join(map(lambda feature: str(int(feature)), features)))

def to_bin(value, bins):
    return numpy.digitize(x=[value], bins=bins)[0]

In [5]:
env = gym.make('CartPole-v0')

env = gym.wrappers.Monitor(env, '/tmp/cartpole-experiment-1', force=True)
    # video_callable=lambda count: count % 10 == 0)

goal_average_steps = 195
max_number_of_steps = 200
last_time_steps = numpy.ndarray(0)
n_bins = 8
n_bins_angle = 10

number_of_features = env.observation_space.shape[0]
last_time_steps = numpy.ndarray(0)

# Number of states is huge so in order to simplify the situation
# we discretize the space to: 10 ** number_of_features
cart_position_bins = pandas.cut([-2.4, 2.4], bins=n_bins, retbins=True)[1][1:-1]
pole_angle_bins = pandas.cut([-2, 2], bins=n_bins_angle, retbins=True)[1][1:-1]
cart_velocity_bins = pandas.cut([-1, 1], bins=n_bins, retbins=True)[1][1:-1]
angle_rate_bins = pandas.cut([-3.5, 3.5], bins=n_bins_angle, retbins=True)[1][1:-1]

# The Q-learn algorithm
qlearn = QLearn(actions=range(env.action_space.n),
                alpha=0.5, gamma=0.90, epsilon=0.1)

for i_episode in xrange(3000):
    observation = env.reset()

    cart_position, pole_angle, cart_velocity, angle_rate_of_change = observation
    state = build_state([to_bin(cart_position, cart_position_bins),
                     to_bin(pole_angle, pole_angle_bins),
                     to_bin(cart_velocity, cart_velocity_bins),
                     to_bin(angle_rate_of_change, angle_rate_bins)])

    for t in xrange(max_number_of_steps):
        # env.render()

        # Pick an action based on the current state
        action = qlearn.chooseAction(state)
        # Execute the action and get feedback
        observation, reward, done, info = env.step(action)

        # Digitize the observation to get a state
        cart_position, pole_angle, cart_velocity, angle_rate_of_change = observation
        nextState = build_state([to_bin(cart_position, cart_position_bins),
                         to_bin(pole_angle, pole_angle_bins),
                         to_bin(cart_velocity, cart_velocity_bins),
                         to_bin(angle_rate_of_change, angle_rate_bins)])

        # # If out of bounds
        # if (cart_position > 2.4 or cart_position < -2.4):
        #     reward = -200
        #     qlearn.learn(state, action, reward, nextState)
        #     print("Out of bounds, reseting")
        #     break

        if not(done):
            qlearn.learn(state, action, reward, nextState)
            state = nextState
        else:
            # Q-learn stuff
            reward = -200
            qlearn.learn(state, action, reward, nextState)
            last_time_steps = numpy.append(last_time_steps, [int(t + 1)])
            break

l = last_time_steps.tolist()
l.sort()
print("Overall score: {:0.2f}".format(last_time_steps.mean()))
print("Best 100 score: {:0.2f}".format(reduce(lambda x, y: x + y, l[-100:]) / len(l[-100:])))

env.close()
# gym.upload('/tmp/cartpole-experiment-1', algorithm_id='vmayoral simple Q-learning', api_key='your-key')

[2017-10-03 10:55:28,780] Making new env: CartPole-v0
[2017-10-03 10:55:28,793] Finished writing results. You can upload them to the scoreboard via gym.upload('/tmp/cartpole-experiment-1')
[2017-10-03 10:55:28,797] Clearing 27 monitor files from previous run (because force=True was provided)
[2017-10-03 10:55:28,807] Starting new video recorder writing to /tmp/cartpole-experiment-1/openaigym.video.2.11868.video000000.mp4
[2017-10-03 10:55:29,436] Starting new video recorder writing to /tmp/cartpole-experiment-1/openaigym.video.2.11868.video000001.mp4
[2017-10-03 10:55:29,674] Starting new video recorder writing to /tmp/cartpole-experiment-1/openaigym.video.2.11868.video000008.mp4
[2017-10-03 10:55:29,920] Starting new video recorder writing to /tmp/cartpole-experiment-1/openaigym.video.2.11868.video000027.mp4
[2017-10-03 10:55:33,369] Starting new video recorder writing to /tmp/cartpole-experiment-1/openaigym.video.2.11868.video000064.mp4
[2017-10-03 10:55:36,679] Starting new video re

Overall score: 179.29
Best 100 score: 200.00


In [ ]:
'''
Deep Q-learning approach to the cartpole problem
using OpenAI's gym environment.

As part of the basic series on reinforcement learning @
https://github.com/vmayoral/basic_reinforcement_learning

This code implements the algorithm described at:
Mnih, V., Kavukcuoglu, K., Silver, D., Rusu, A. A., Veness, J., Bellemare, M. G., ... & Petersen, 
S. (2015). Human-level control through deep reinforcement learning. Nature, 518(7540), 529-533.

Code based on @wingedsheep's work at https://gist.github.com/wingedsheep/4199594b02138dd427c22a540d6d6b8d

        @author: Victor Mayoral Vilches <victor@erlerobotics.com>
'''

import gym
import random
import numpy as np
from keras.models import Sequential
from keras import optimizers
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.regularizers import l2

# import os
# os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"
# import theano

class Memory:
    """
    This class provides an abstraction to store the [s, a, r, a'] elements of each iteration.
    Instead of using tuples (as other implementations do), the information is stored in lists 
    that get returned as another list of dictionaries with each key corresponding to either 
    "state", "action", "reward", "nextState" or "isFinal".
    """
    def __init__(self, size):
        self.size = size
        self.currentPosition = 0
        self.states = []
        self.actions = []
        self.rewards = []
        self.newStates = []
        self.finals = []

    def getMiniBatch(self, size) :
        indices = random.sample(np.arange(len(self.states)), min(size,len(self.states)) )
        miniBatch = []
        for index in indices:
            miniBatch.append({'state': self.states[index],'action': self.actions[index], 'reward': self.rewards[index], 'newState': self.newStates[index], 'isFinal': self.finals[index]})
        return miniBatch

    def getCurrentSize(self) :
        return len(self.states)

    def getMemory(self, index): 
        return {'state': self.states[index],'action': self.actions[index], 'reward': self.rewards[index], 'newState': self.newStates[index], 'isFinal': self.finals[index]}

    def addMemory(self, state, action, reward, newState, isFinal) :
        if (self.currentPosition >= self.size - 1) :
            self.currentPosition = 0
        if (len(self.states) > self.size) :
            self.states[self.currentPosition] = state
            self.actions[self.currentPosition] = action
            self.rewards[self.currentPosition] = reward
            self.newStates[self.currentPosition] = newState
            self.finals[self.currentPosition] = isFinal
        else :
            self.states.append(state)
            self.actions.append(action)
            self.rewards.append(reward)
            self.newStates.append(newState)
            self.finals.append(isFinal)
        
        self.currentPosition += 1

class DeepQ:
    """
    DQN abstraction.

    As a quick reminder:
        traditional Q-learning:
            Q(s, a) += alpha * (reward(s,a) + gamma * max(Q(s') - Q(s,a))
        DQN:
            target = reward(s,a) + gamma * max(Q(s')

    """
    def __init__(self, inputs, outputs, memorySize, discountFactor, learningRate, learnStart):
        """
        Parameters:
            - inputs: input size
            - outputs: output size
            - memorySize: size of the memory that will store each state
            - discountFactor: the discount factor (gamma)
            - learningRate: learning rate
            - learnStart: steps to happen before for learning. Set to 128
        """
        self.input_size = inputs
        self.output_size = outputs
        self.memory = Memory(memorySize)
        self.discountFactor = discountFactor
        self.learnStart = learnStart
        self.learningRate = learningRate
   
    def initNetworks(self, hiddenLayers):
        model = self.createModel(self.input_size, self.output_size, hiddenLayers, "relu", self.learningRate)
        self.model = model

        targetModel = self.createModel(self.input_size, self.output_size, hiddenLayers, "relu", self.learningRate)
        self.targetModel = targetModel

    def createRegularizedModel(self, inputs, outputs, hiddenLayers, activationType, learningRate):
        bias = True
        dropout = 0
        regularizationFactor = 0.01
        model = Sequential()
        if len(hiddenLayers) == 0: 
            model.add(Dense(self.output_size, input_shape=(self.input_size,), init='lecun_uniform', bias=bias))
            model.add(Activation("linear"))
        else :
            if regularizationFactor > 0:
                model.add(Dense(hiddenLayers[0], input_shape=(self.input_size,), init='lecun_uniform', W_regularizer=l2(regularizationFactor),  bias=bias))
            else:
                model.add(Dense(hiddenLayers[0], input_shape=(self.input_size,), init='lecun_uniform', bias=bias))

            if (activationType == "LeakyReLU") :
                model.add(LeakyReLU(alpha=0.01))
            else :
                model.add(Activation(activationType))
            
            for index in range(1, len(hiddenLayers)):
                layerSize = hiddenLayers[index]
                if regularizationFactor > 0:
                    model.add(Dense(layerSize, init='lecun_uniform', W_regularizer=l2(regularizationFactor), bias=bias))
                else:
                    model.add(Dense(layerSize, init='lecun_uniform', bias=bias))
                if (activationType == "LeakyReLU") :
                    model.add(LeakyReLU(alpha=0.01))
                else :
                    model.add(Activation(activationType))
                if dropout > 0:
                    model.add(Dropout(dropout))
            model.add(Dense(self.output_size, init='lecun_uniform', bias=bias))
            model.add(Activation("linear"))
        optimizer = optimizers.RMSprop(lr=learningRate, rho=0.9, epsilon=1e-06)
        model.compile(loss="mse", optimizer=optimizer)
        model.summary()
        return model

    def createModel(self, inputs, outputs, hiddenLayers, activationType, learningRate):
        model = Sequential()
        if len(hiddenLayers) == 0: 
            model.add(Dense(self.output_size, input_shape=(self.input_size,), init='lecun_uniform'))
            model.add(Activation("linear"))
        else :
            model.add(Dense(hiddenLayers[0], input_shape=(self.input_size,), init='lecun_uniform'))
            if (activationType == "LeakyReLU") :
                model.add(LeakyReLU(alpha=0.01))
            else :
                model.add(Activation(activationType))
            
            for index in range(1, len(hiddenLayers)):
                # print("adding layer "+str(index))
                layerSize = hiddenLayers[index]
                model.add(Dense(layerSize, init='lecun_uniform'))
                if (activationType == "LeakyReLU") :
                    model.add(LeakyReLU(alpha=0.01))
                else :
                    model.add(Activation(activationType))
            model.add(Dense(self.output_size, init='lecun_uniform'))
            model.add(Activation("linear"))
        optimizer = optimizers.RMSprop(lr=learningRate, rho=0.9, epsilon=1e-06)
        model.compile(loss="mse", optimizer=optimizer)
        model.summary()
        return model

    def printNetwork(self):
        i = 0
        for layer in self.model.layers:
            weights = layer.get_weights()
            print "layer ",i,": ",weights
            i += 1


    def backupNetwork(self, model, backup):
        weightMatrix = []
        for layer in model.layers:
            weights = layer.get_weights()
            weightMatrix.append(weights)
        i = 0
        for layer in backup.layers:
            weights = weightMatrix[i]
            layer.set_weights(weights)
            i += 1

    def updateTargetNetwork(self):
        self.backupNetwork(self.model, self.targetModel)

    # predict Q values for all the actions
    def getQValues(self, state):
        predicted = self.model.predict(state.reshape(1,len(state)))
        return predicted[0]

    def getTargetQValues(self, state):
        predicted = self.targetModel.predict(state.reshape(1,len(state)))
        return predicted[0]

    def getMaxQ(self, qValues):
        return np.max(qValues)

    def getMaxIndex(self, qValues):
        return np.argmax(qValues)

    # calculate the target function
    def calculateTarget(self, qValuesNewState, reward, isFinal):
        """
        target = reward(s,a) + gamma * max(Q(s')
        """
        if isFinal:
            return reward
        else : 
            return reward + self.discountFactor * self.getMaxQ(qValuesNewState)

    # select the action with the highest Q value
    def selectAction(self, qValues, explorationRate):
        rand = random.random()
        if rand < explorationRate :
            action = np.random.randint(0, self.output_size)
        else :
            action = self.getMaxIndex(qValues)
        return action

    def selectActionByProbability(self, qValues, bias):
        qValueSum = 0
        shiftBy = 0
        for value in qValues:
            if value + shiftBy < 0:
                shiftBy = - (value + shiftBy)
        shiftBy += 1e-06

        for value in qValues:
            qValueSum += (value + shiftBy) ** bias

        probabilitySum = 0
        qValueProbabilities = []
        for value in qValues:
            probability = ((value + shiftBy) ** bias) / float(qValueSum)
            qValueProbabilities.append(probability + probabilitySum)
            probabilitySum += probability
        qValueProbabilities[len(qValueProbabilities) - 1] = 1

        rand = random.random()
        i = 0
        for value in qValueProbabilities:
            if (rand <= value):
                return i
            i += 1

    def addMemory(self, state, action, reward, newState, isFinal):
        self.memory.addMemory(state, action, reward, newState, isFinal)

    def learnOnLastState(self):
        if self.memory.getCurrentSize() >= 1:
            return self.memory.getMemory(self.memory.getCurrentSize() - 1)

    def learnOnMiniBatch(self, miniBatchSize, useTargetNetwork=True):
        # Do not learn until we've got self.learnStart samples        
        if self.memory.getCurrentSize() > self.learnStart:
            # learn in batches of 128
            miniBatch = self.memory.getMiniBatch(miniBatchSize)
            X_batch = np.empty((0,self.input_size), dtype = np.float64)
            Y_batch = np.empty((0,self.output_size), dtype = np.float64)
            for sample in miniBatch:
                isFinal = sample['isFinal']
                state = sample['state']
                action = sample['action']
                reward = sample['reward']
                newState = sample['newState']

                qValues = self.getQValues(state)
                if useTargetNetwork:
                    qValuesNewState = self.getTargetQValues(newState)
                else :
                    qValuesNewState = self.getQValues(newState)
                targetValue = self.calculateTarget(qValuesNewState, reward, isFinal)

                X_batch = np.append(X_batch, np.array([state.copy()]), axis=0)
                Y_sample = qValues.copy()
                Y_sample[action] = targetValue
                Y_batch = np.append(Y_batch, np.array([Y_sample]), axis=0)
                if isFinal:
                    X_batch = np.append(X_batch, np.array([newState.copy()]), axis=0)
                    Y_batch = np.append(Y_batch, np.array([[reward]*self.output_size]), axis=0)
            self.model.fit(X_batch, Y_batch, batch_size = len(miniBatch), nb_epoch=1, verbose = 0)

env = gym.make('CartPole-v0')

epochs = 1000
steps = 100000
updateTargetNetwork = 10000
explorationRate = 1
minibatch_size = 128
learnStart = 128
learningRate = 0.00025
discountFactor = 0.99
memorySize = 1000000

last100Scores = [0] * 100
last100ScoresIndex = 0
last100Filled = False

deepQ = DeepQ(4, 2, memorySize, discountFactor, learningRate, learnStart)
# deepQ.initNetworks([30,30,30])
# deepQ.initNetworks([30,30])
deepQ.initNetworks([300,300])

stepCounter = 0

# number of reruns
for epoch in xrange(epochs):
    observation = env.reset()
    print explorationRate
    # number of timesteps
    for t in xrange(steps):
        # env.render()
        qValues = deepQ.getQValues(observation)

        action = deepQ.selectAction(qValues, explorationRate)

        newObservation, reward, done, info = env.step(action)

        if (t >= 199):
            print "reached the end! :D"
            done = True
            # reward = 200            

        if done and t < 199:
            print "decrease reward"
            # reward -= 200
        deepQ.addMemory(observation, action, reward, newObservation, done)

        if stepCounter >= learnStart:
            if stepCounter <= updateTargetNetwork:
                deepQ.learnOnMiniBatch(minibatch_size, False)
            else :
                deepQ.learnOnMiniBatch(minibatch_size, True)

        observation = newObservation

        if done:
            last100Scores[last100ScoresIndex] = t
            last100ScoresIndex += 1
            if last100ScoresIndex >= 100:
                last100Filled = True
                last100ScoresIndex = 0
            if not last100Filled:
                print "Episode ",epoch," finished after {} timesteps".format(t+1)
            else :
                print "Episode ",epoch," finished after {} timesteps".format(t+1)," last 100 average: ",(sum(last100Scores)/len(last100Scores))
            break

        stepCounter += 1
        if stepCounter % updateTargetNetwork == 0:
            deepQ.updateTargetNetwork()
            print "updating target network"

    explorationRate *= 0.995
    # explorationRate -= (2.0/epochs)
    explorationRate = max (0.05, explorationRate)

Using TensorFlow backend.
[2017-10-03 12:47:01,080] Making new env: CartPole-v0
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:157: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(300, kernel_initializer="lecun_uniform", input_shape=(4,))`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:166: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(300, kernel_initializer="lecun_uniform")`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:171: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_initializer="lecun_uniform")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 300)               1500      
_________________________________________________________________
activation_1 (Activation)    (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 300)               90300     
_________________________________________________________________
activation_2 (Activation)    (None, 300)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 602       
_________________________________________________________________
activation_3 (Activation)    (None, 2)                 0         
Total params: 92,402
Trainable params: 92,402
Non-trainable params: 0
_________________________________________________________________
______

/usr/local/lib/python2.7/dist-packages/keras/models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


decrease reward
Episode  7  finished after 12 timesteps
0.960693043575
decrease reward
Episode  8  finished after 14 timesteps
0.955889578358
decrease reward
Episode  9  finished after 12 timesteps
0.951110130466
decrease reward
Episode  10  finished after 10 timesteps
0.946354579813
decrease reward
Episode  11  finished after 20 timesteps
0.941622806914
decrease reward
Episode  12  finished after 16 timesteps
0.93691469288
decrease reward
Episode  13  finished after 18 timesteps
0.932230119415
decrease reward
Episode  14  finished after 13 timesteps
0.927568968818
decrease reward
Episode  15  finished after 15 timesteps
0.922931123974
decrease reward
Episode  16  finished after 28 timesteps
0.918316468354
decrease reward
Episode  17  finished after 42 timesteps
0.913724886013
decrease reward
Episode  18  finished after 15 timesteps
0.909156261583
decrease reward
Episode  19  finished after 13 timesteps
0.904610480275
decrease reward
Episode  20  finished after 15 timesteps
0.900087427

decrease reward
Episode  116  finished after 20 timesteps  last 100 average:  49
0.556288967872
reached the end! :D
Episode  117  finished after 200 timesteps  last 100 average:  51
0.553507523032
decrease reward
Episode  118  finished after 51 timesteps  last 100 average:  51
0.550739985417
decrease reward
Episode  119  finished after 177 timesteps  last 100 average:  53
0.54798628549
decrease reward
Episode  120  finished after 35 timesteps  last 100 average:  53
0.545246354063
decrease reward
Episode  121  finished after 29 timesteps  last 100 average:  53
0.542520122292
decrease reward
Episode  122  finished after 17 timesteps  last 100 average:  53
0.539807521681
decrease reward
Episode  123  finished after 137 timesteps  last 100 average:  54
0.537108484072
decrease reward
Episode  124  finished after 135 timesteps  last 100 average:  55
0.534422941652
decrease reward
Episode  125  finished after 153 timesteps  last 100 average:  57
0.531750826944
decrease reward
Episode  126  fi

decrease reward
Episode  200  finished after 150 timesteps  last 100 average:  115
0.365123032618
decrease reward
Episode  201  finished after 184 timesteps  last 100 average:  115
0.363297417454
reached the end! :D
Episode  202  finished after 200 timesteps  last 100 average:  116
0.361480930367
decrease reward
Episode  203  finished after 141 timesteps  last 100 average:  118
0.359673525715
reached the end! :D
Episode  204  finished after 200 timesteps  last 100 average:  119
0.357875158087
decrease reward
Episode  205  finished after 193 timesteps  last 100 average:  120
0.356085782296
decrease reward
Episode  206  finished after 122 timesteps  last 100 average:  121
0.354305353385
decrease reward
Episode  207  finished after 125 timesteps  last 100 average:  122
0.352533826618
decrease reward
Episode  208  finished after 160 timesteps  last 100 average:  123
0.350771157485
decrease reward
Episode  209  finished after 67 timesteps  last 100 average:  123
0.349017301697
decrease rewa

decrease reward
Episode  283  finished after 165 timesteps  last 100 average:  160
0.240854592576
decrease reward
Episode  284  finished after 165 timesteps  last 100 average:  161
0.239650319613
reached the end! :D
Episode  285  finished after 200 timesteps  last 100 average:  162
0.238452068015
updating target network
reached the end! :D
Episode  286  finished after 200 timesteps  last 100 average:  162
0.237259807675
decrease reward
Episode  287  finished after 166 timesteps  last 100 average:  162
0.236073508637
decrease reward
Episode  288  finished after 163 timesteps  last 100 average:  162
0.234893141094
decrease reward
Episode  289  finished after 150 timesteps  last 100 average:  162
0.233718675388
decrease reward
Episode  290  finished after 181 timesteps  last 100 average:  162
0.232550082011
decrease reward
Episode  291  finished after 184 timesteps  last 100 average:  162
0.231387331601
reached the end! :D
Episode  292  finished after 200 timesteps  last 100 average:  164

decrease reward
Episode  365  finished after 174 timesteps  last 100 average:  183
0.159678907633
decrease reward
Episode  366  finished after 177 timesteps  last 100 average:  183
0.158880513095
